In [ ]:
import pandas as pd

from accelerate import Accelerator

In [ ]:
from config.config import CONFIG

In [ ]:
from src.models.embedder import Embedder
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(CONFIG['model'], use_fast=CONFIG['tokenizer_use_fast'])
embedder = Embedder(CONFIG).to(CONFIG['device'])

In [ ]:
train_df = pd.read_csv(CONFIG['train_path'])
dev_df = pd.read_csv(CONFIG['dev_path'])

In [ ]:
from src.models.utils import query_embedding

train_df['query_embed'] = train_df['query'].apply(lambda x: query_embedding(x, embedder, tokenizer, CONFIG))
dev_df['query_embed'] = dev_df['query'].apply(lambda x: query_embedding(x, embedder, tokenizer, CONFIG))

In [ ]:
def upsample_data(df):
    # Upsample the data to balance across languages
    lang_counts = df['lang'].value_counts()
    max_count = lang_counts.max()
    upsampled_dfs = []
    for lang in lang_counts.index:
        lang_df = df[df['lang'] == lang]
        upsampled_df = lang_df.sample(max_count - lang_counts[lang], replace=True)
        upsampled_dfs.append(pd.concat([lang_df, upsampled_df]))
    return pd.concat(upsampled_dfs)

In [ ]:
train_df = upsample_data(train_df)

In [ ]:
# save query embeddings
train_df.to_csv(CONFIG['train_emb_path'], index=False)
dev_df.to_csv(CONFIG['dev_emb_path'], index=False)

In [ ]:
# load query embeddings
train_df = pd.read_csv(CONFIG['train_emb_path'])
dev_df = pd.read_csv(CONFIG['dev_emb_path'])

In [ ]:
import pickle

# load document embeddings
with open(CONFIG['doc_embeds_path'], 'rb') as f:
    doc_embeds = pickle.load(f)

In [ ]:
from src.dataloaders.utils import get_train_val_dataloaders

train_dl, val_dl = get_train_val_dataloaders(CONFIG, train_df, dev_df, doc_embeds)

In [ ]:
accelerator = Accelerator(gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'])

In [ ]:
from src.training.trainner import Trainer
from src.models.dpr import DPRModel

embed_size = embedder.model.get_output_dim()
model = DPRModel(embed_size).to(CONFIG['device'])
trainer = Trainer(model, (train_dl, val_dl), CONFIG, accelerator)

In [ ]:
trainer.train()

In [ ]:
losses_df = pd.DataFrame({'epoch':list(range(1, CONFIG['epochs'] + 1)),
                          'train_loss':trainer.train_losses, 
                          'val_loss': trainer.val_losses
                         })
losses_df.to_csv(CONFIG['losses_path'], index=False)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(trainer.train_losses, color='red')
plt.plot(trainer.val_losses, color='orange')
plt.title('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')